In [8]:
!pip install streamlit pyngrok


In [9]:
from pyngrok import ngrok

# Replace YOUR_TOKEN_HERE with your real authtoken
!ngrok config add-authtoken 2zVNUyRBy7HJLEJKnmYw9e6nhZm_6moVuJ9bHtos9fNcL3t4k


Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [10]:
%%writefile app.py
import streamlit as st
import pandas as pd
import joblib

# Load model
model = joblib.load("model.pkl")

# Expected order from model
expected_columns = [
    'Age', 'Sex', 'Job', 'Housing',
    'Saving accounts', 'Checking account',
    'Credit amount', 'Duration', 'Purpose'
]

# Streamlit UI
st.set_page_config(page_title="Credit Risk Classifier", layout="centered")
st.title("💳 Credit Risk Prediction")

# UI Inputs
age = st.slider("Age", 18, 75, 30)
sex = st.selectbox("Sex", ['male', 'female'])
job = st.selectbox("Job Type", [0, 1, 2, 3])
housing = st.selectbox("Housing", ['own', 'rent', 'free'])
saving = st.selectbox("Saving accounts", ['little', 'moderate', 'quite rich', 'rich', 'unknown'])
checking = st.selectbox("Checking account", ['<0', '0<=X<200', '>=200', 'unknown'])
credit = st.number_input("Credit amount", min_value=0, value=1000)
duration = st.slider("Duration (months)", 4, 72, 12)
purpose = st.selectbox("Purpose", [
    'car', 'furniture/equipment', 'radio/TV', 'domestic appliances',
    'repairs', 'education', 'business', 'vacation/others'
])

if st.button("🔍 Predict Credit Risk"):
    input_dict = {
        'Age': age,
        'Sex': sex,
        'Job': job,
        'Housing': housing,
        'Saving accounts': saving,
        'Checking account': checking,
        'Credit amount': credit,
        'Duration': duration,
        'Purpose': purpose
    }

    input_df = pd.DataFrame([input_dict])

    encoding_maps = {
        'Sex': {'male': 1, 'female': 0},
        'Housing': {'own': 2, 'rent': 1, 'free': 0},
        'Saving accounts': {'little': 2, 'moderate': 3, 'quite rich': 4, 'rich': 1, 'unknown': 0},
        'Checking account': {'<0': 1, '0<=X<200': 2, '>=200': 3, 'unknown': 0},
        'Purpose': {
            'car': 0, 'furniture/equipment': 1, 'radio/TV': 6, 'domestic appliances': 2,
            'repairs': 5, 'education': 3, 'business': 4, 'vacation/others': 7
        }
    }

    for col, mapping in encoding_maps.items():
        input_df[col] = input_df[col].map(mapping)

    input_df = input_df[expected_columns]
    input_df.columns = model.feature_names_in_
    input_df = input_df[model.feature_names_in_]  # enforce correct order

    prediction = model.predict(input_df)[0]
    proba = model.predict_proba(input_df)[0][prediction]

    result = "✅ Good Credit Risk" if prediction == 0 else "❌ Bad Credit Risk"
    st.subheader("Result")
    st.success(f"{result}\nConfidence: {round(proba * 100, 2)}%")


Overwriting app.py


In [11]:
!streamlit run app.py &>/dev/null &

# Open tunnel
public_url = ngrok.connect(8501)
print("🌐 Your Streamlit app is live at:", public_url)

🌐 Your Streamlit app is live at: NgrokTunnel: "https://1fcb-35-192-196-58.ngrok-free.app" -> "http://localhost:8501"


In [ ]:
#0 means good risk can give the credit
#1 means bad risk and cannot give
#There’s a 67.36% probability that the applicant truly belongs to the "good credit" class (as per the model's learned patterns).